In [1]:
import os
import sys
from google.colab import drive
import glob
from pathlib import Path
import shutil
from tqdm.auto import tqdm
def make_dirs():
    # make dirs for models & logs
    dirs = [
        "/content/models",

        "/content/models/segmentation",
        "/content/models/classification",
        "/content/models/autoencoder",

        "/content/models/segmentation/unet",
        "/content/models/segmentation/segformer",
        "/content/models/classification/efficienet",
        "/content/models/classification/convnext",

        "/content/models_log",

        "/content/models_log/segmentation",
        "/content/models_log/segmentation/unet",
        "/content/models_log/segmentation/segformer",

        "/content/models_log/classification",
        "/content/models_log/classification/efficienet",
        "/content/models_log/classification/convnext",
        "/content/models_log/autoencoder",
    ]
    for dir in dirs:
        os.makedirs(dir, exist_ok=True)



def mount_drive():

    drive.mount('/content/drive')
    source = '/content/drive/MyDrive/datasets/multi'  # Na Drive
    destination = '/content/datasets/multi'            # Link lokalny

    os.makedirs('/content/datasets', exist_ok=True)

    if os.path.exists(destination):
        if os.path.islink(destination):
            os.unlink(destination)
        else:
            shutil.rmtree(destination)

    # os.symlink(source, destination)
    print(f"Copying dataset from {source} to {destination}...")

    # Count files for progress bar
    total_files = sum([len(files) for r, d, files in os.walk(source)])

    with tqdm(total=total_files, desc="Copying", unit="file") as pbar:
        def copy_func(src, dst):
            shutil.copy2(src, dst)
            pbar.update(1)

        shutil.copytree(source, destination, copy_function=copy_func)

    print(f"Dataset copied!")
    print(f"  {destination} <- {source}")

def set_colab(path: str, on_colab: bool):
    dir = Path(path)
    result = list(dir.rglob("hparams.[pP][yY]"))
    for r in result:
        with open(r, "r+") as f:
            pos = f.tell()
            line = f.readline()
            if on_colab:
                if line == "ON_COLAB = False\n":
                    f.seek(pos)
                    f.write("ON_COLAB = True\n")
            else:
                if line == "ON_COLAB = True\n":
                    f.seek(pos)
                    f.write("ON_COLAB = False\n")



In [2]:
!sudo apt-get -qq install zstd
!gdown --id 1ZCY_ewakkoxohu4qqk4kHHgBCqi0urIn -O dataset_segmentation.tar.zst
!mkdir /content/datasets
!tar -xf /content/dataset_segmentation.tar.zst -C /content/datasets/
!mv /content/datasets/dataset_segmentation/ /content/datasets/multi/



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package zstd.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (o

In [3]:
!pip install -q segmentation-models-pytorch torchmetrics
!pip install -q torchmetrics
!git clone -q https://github.com/mszczesniak02/bachlor_backup.git
make_dirs()
set_colab("/content/bachlor_backup/src", True)

!cat bachlor_backup/src/segmentation/common/hparams.py | head -1
!cat bachlor_backup/src/classification/common/hparams.py | head -1
!cat bachlor_backup/src/autoencoder/hparams.py | head -1
!cat bachlor_backup/src/dim_translator/hparams.py | head -1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 62.6 MB/s eta 0:00:00
ON_COLAB = True
ON_COLAB = True
ON_COLAB = True
cat: bachlor_backup/src/dim_translator/hparams.py: No such file or directory


In [20]:
!rm -rd bachlor_backup/
!git clone -q https://github.com/mszczesniak02/bachlor_backup.git
make_dirs()
set_colab("/content/bachlor_backup/src", True)

!cat bachlor_backup/src/segmentation/common/hparams.py | head -1
!cat bachlor_backup/src/classification/common/hparams.py | head -1
!cat bachlor_backup/src/autoencoder/hparams.py | head -1
!cat bachlor_backup/src/dim_translator/hparams.py | head -1


ON_COLAB = True
ON_COLAB = True
ON_COLAB = True
cat: bachlor_backup/src/dim_translator/hparams.py: No such file or directory


In [ ]:
!cd bachlor_backup/src/segmentation/unet/ && python train.py

In [4]:

original_sys_path = sys.path.copy()

# sys.path.append('/content/bachlor_backup/src/autoencoder')
# import train as autoencoder_train

sys.path.append('/content/bachlor_backup/src/segmentation/unet/')
import train as unet_train

# sys.path.append('/content/bachlor_backup/src/segmentation/segformer/')
# import train as segformer_train

# sys.path.append('/content/bachlor_backup/src/classification/convnext')
# import train as convnext_train

# sys.path.append('/content/bachlor_backup/src/classification/efficienet')
# import train as efficienet_train


In [5]:
unet_train.main()

Seeding with 42
Dataloader initialized.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

Criterion initialized: Dice BCE Loss
Optimizer initialized: Adam
Scheduler initialized: ReduceLROnPlateau


Epochs:   0%|          | 0/25 [00:00<?, ?it/s]

Training:   0%|          | 0/620 [00:00<?, ?it/s]

Validation:   0%|          | 0/107 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7bfbfbf80900>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7bfbfbf80900>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 16

KeyError: 'accuracy'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')